In [255]:
from scipy import io
import pandas as pd
import os
from tqdm import tqdm
import numpy as np
from pathlib import Path
from itertools import combinations
import random
import pathlib
from count_doublets_utils import *
np.random.seed(2022)

In [125]:
test_sample = "/projects/p31666/zzhang/doublet-bchmk/data/fatemap_data/FM01/fatemapID/stepFourStarcodeShavedReads50.txt"
umi_cutoff_ratio=3 / 4e5
umi_diff_threshold=50
dominant_threshold=10
num_tests=5000
data_root = "/projects/p31666/zzhang/doublet-bchmk/data/fatemap_data"
keyword = "stepFourStarcodeShavedReads50"
pattern = '**/{}*'.format(keyword)
def grab_input_files(root_dir, file_format):
    grabbed_files = []
    for filepath in pathlib.Path(root_dir).glob(pattern):
        grabbed_files.append(str(filepath.absolute()))
    return grabbed_files

## Test on if good data is correct 

In [127]:
all_fatemap_read_files = grab_input_files(data_root, pattern)
for cur_fatemap_read_file in all_fatemap_read_files:
    print("INFO: Testing on {}".format(cur_fatemap_read_file))
    df = pd.read_csv(cur_fatemap_read_file, sep='\t')
    header = list(df.columns)
    if header[-1].lower() == "sampleNum".lower():
        header[-1] = "sampleNum"
        df.columns = header
    good_data_ls = []
    for cur_sample_num in df["sampleNum"].unique():
        cur_sample_df = df[df["sampleNum"] == cur_sample_num]
        cur_umi_adjusted_cutoff = round(umi_cutoff_ratio * cur_sample_df.shape[0])
        print("Current Sample Adjusted UMI cutoff: {}".format(cur_umi_adjusted_cutoff))
        cur_freq_df = cur_sample_df.groupby(['cellID', 'BC50StarcodeD8', 'sampleNum']).size().reset_index()
        cur_freq_df.columns = ['cellID', "fatemapID", 'sampleNum', "nUMI"]
        cur_good_data = cur_freq_df[cur_freq_df['nUMI'] >= cur_umi_adjusted_cutoff].reset_index(drop=True)
        good_data_ls.append(cur_good_data)

    good_data = pd.concat(good_data_ls)
    good_data = good_data.sort_values(by=["sampleNum", 'cellID', 'fatemapID', 'nUMI']).reset_index(drop=True)
    idx_list = random.sample(list(np.arange(good_data.shape[0])), num_tests)
    umi_cutoff_dict = {}
    for cur_sample_num in df["sampleNum"].unique():
        cur_sample_df = df[df["sampleNum"] == cur_sample_num]
        cur_umi_adjusted_cutoff = round(umi_cutoff_ratio * cur_sample_df.shape[0])
        umi_cutoff_dict[cur_sample_num]=cur_umi_adjusted_cutoff
    for idx in tqdm(idx_list):
        cur_sample = good_data.iloc[idx]["sampleNum"]
        cell_id = good_data.iloc[idx]["cellID"]
        fatemap_id = good_data.iloc[idx]["fatemapID"]
        cur_df = df[(df["cellID"]==cell_id) & (df["BC50StarcodeD8"]==fatemap_id) & (df["sampleNum"]==cur_sample)]
        correct_umi_count=cur_df.shape[0]
        if(correct_umi_count<umi_cutoff_dict[cur_sample]):
            continue
        calculated_count = good_data[(good_data["cellID"]==cell_id) & \
                                     (good_data["fatemapID"]==fatemap_id) & (good_data["sampleNum"]==cur_sample)]["nUMI"].values[0]
        if(correct_umi_count!=calculated_count):
            break

INFO: Testing on /projects/p31666/zzhang/doublet-bchmk/data/fatemap_data/FM04/fatemapID/stepFourStarcodeShavedReads50.txt
Current Sample Adjusted UMI cutoff: 2
Current Sample Adjusted UMI cutoff: 3


100%|██████████| 5000/5000 [06:28<00:00, 12.87it/s]


INFO: Testing on /projects/p31666/zzhang/doublet-bchmk/data/fatemap_data/non_cancer/fatemapID/stepFourStarcodeShavedReads50.txt
Current Sample Adjusted UMI cutoff: 4
Current Sample Adjusted UMI cutoff: 2


100%|██████████| 5000/5000 [07:02<00:00, 11.83it/s]


INFO: Testing on /projects/p31666/zzhang/doublet-bchmk/data/fatemap_data/FM05/fatemapID/stepFourStarcodeShavedReads50.txt
Current Sample Adjusted UMI cutoff: 2
Current Sample Adjusted UMI cutoff: 2
Current Sample Adjusted UMI cutoff: 1
Current Sample Adjusted UMI cutoff: 1


...

100%|██████████| 5000/5000 [48:23<00:00,  1.72it/s]


INFO: Testing on /projects/p31666/zzhang/doublet-bchmk/data/fatemap_data/FM03/fatemapID/stepFourStarcodeShavedReads50.txt
Current Sample Adjusted UMI cutoff: 12
Current Sample Adjusted UMI cutoff: 12


100%|██████████| 5000/5000 [28:04<00:00,  2.97it/s]


INFO: Testing on /projects/p31666/zzhang/doublet-bchmk/data/fatemap_data/FM08/fatemapID/stepFourStarcodeShavedReads50.txt
Current Sample Adjusted UMI cutoff: 5
Current Sample Adjusted UMI cutoff: 11
Current Sample Adjusted UMI cutoff: 0


100%|██████████| 5000/5000 [17:14<00:00,  4.83it/s]


## Test on if fatemap barcode to cellID dict is produced correctly

In [170]:
num_tests=5000

In [171]:
all_fatemap_read_files = grab_input_files(data_root, pattern)

for cur_fatemap_read_file in all_fatemap_read_files:
    print("INFO: Testing on {}".format(cur_fatemap_read_file))
    df = pd.read_csv(cur_fatemap_read_file, sep='\t')
    header = list(df.columns)
    if header[-1].lower() == "sampleNum".lower():
        header[-1] = "sampleNum"
        df.columns = header
    good_data_ls = []
    for cur_sample_num in df["sampleNum"].unique():
        cur_sample_df = df[df["sampleNum"] == cur_sample_num]
        cur_umi_adjusted_cutoff = round(umi_cutoff_ratio * cur_sample_df.shape[0])
        print("Current Sample Adjusted UMI cutoff: {}".format(cur_umi_adjusted_cutoff))
        cur_freq_df = cur_sample_df.groupby(['cellID', 'BC50StarcodeD8', 'sampleNum']).size().reset_index()
        cur_freq_df.columns = ['cellID', "fatemapID", 'sampleNum', "nUMI"]
        cur_good_data = cur_freq_df[cur_freq_df['nUMI'] >= cur_umi_adjusted_cutoff].reset_index(drop=True)
        good_data_ls.append(cur_good_data)

    good_data = pd.concat(good_data_ls)
    good_data = good_data.sort_values(by=["sampleNum", 'cellID', 'fatemapID', 'nUMI']).reset_index(drop=True)

    all_samples = good_data['sampleNum'].unique()
    cellID2fatemap_dict = {}
    cellID2fatemap_count_dict = {}

    # initialize the count dictionaries
    for i in all_samples:
        cellID2fatemap_dict[i] = {}
        cellID2fatemap_count_dict[i] = {}

    # loop through each row
    for index, row in good_data.iterrows():
        cur_cellID = row['cellID']
        cur_fateID = row['fatemapID']
        cur_sample_num = int(row['sampleNum'])

        # only look at samples 1 and 2 for now
        # if(cur_sample_num not in all_samples):
        #     continue
        if cur_cellID not in cellID2fatemap_dict[cur_sample_num].keys():
            cellID2fatemap_dict[cur_sample_num][cur_cellID] = [cur_fateID]
            cellID2fatemap_count_dict[cur_sample_num][cur_cellID] = 1
        else:
            # only add if the fateID is not present in the dict
            if cur_fateID not in cellID2fatemap_dict[cur_sample_num][cur_cellID]:
                cellID2fatemap_dict[cur_sample_num][cur_cellID].append(cur_fateID)
                cellID2fatemap_count_dict[cur_sample_num][cur_cellID] += 1
                
#     dataset_dicts[cur_fatemap_read_file]["cellID2fatemap_dict"]=cellID2fatemap_dict
#     dataset_dicts[cur_fatemap_read_file]["cellID2fatemap_count_dict"]=cellID2fatemap_count_dict
    
    ########################
    ######## TEST ##########
    ########################
    for cur_sample in cellID2fatemap_count_dict.keys():
        cur_sample_cellIDs = cellID2fatemap_count_dict[cur_sample].keys()
        cellIDs_test = random.sample(cur_sample_cellIDs, min(num_tests, len(cur_sample_cellIDs)))
        for cur_cellID in tqdm(cellIDs_test):
            cur_cellID_count = cellID2fatemap_count_dict[cur_sample][cur_cellID]
            cur_cellID_fatemap = cellID2fatemap_dict[cur_sample][cur_cellID]
            cur_df = good_data[(good_data["cellID"]==cur_cellID) & (good_data["sampleNum"]==cur_sample)]
            cur_fatemap_count = cur_df.shape[0]
            if(cur_cellID_count!=cur_fatemap_count):
                break
            if(set(cur_df["fatemapID"].values)!=set(cur_cellID_fatemap)):
                break

INFO: Testing on /projects/p31666/zzhang/doublet-bchmk/data/fatemap_data/FM04/fatemapID/stepFourStarcodeShavedReads50.txt
Current Sample Adjusted UMI cutoff: 2
Current Sample Adjusted UMI cutoff: 3


100%|██████████| 5000/5000 [00:09<00:00, 539.23it/s]


INFO: Testing on /projects/p31666/zzhang/doublet-bchmk/data/fatemap_data/non_cancer/fatemapID/stepFourStarcodeShavedReads50.txt
Current Sample Adjusted UMI cutoff: 4
Current Sample Adjusted UMI cutoff: 2


100%|██████████| 2794/2794 [00:07<00:00, 381.82it/s]


INFO: Testing on /projects/p31666/zzhang/doublet-bchmk/data/fatemap_data/FM05/fatemapID/stepFourStarcodeShavedReads50.txt
Current Sample Adjusted UMI cutoff: 2
Current Sample Adjusted UMI cutoff: 2
Current Sample Adjusted UMI cutoff: 1
Current Sample Adjusted UMI cutoff: 1


...

100%|██████████| 5000/5000 [00:14<00:00, 349.67it/s]


INFO: Testing on /projects/p31666/zzhang/doublet-bchmk/data/fatemap_data/FM03/fatemapID/stepFourStarcodeShavedReads50.txt
Current Sample Adjusted UMI cutoff: 12
Current Sample Adjusted UMI cutoff: 12


100%|██████████| 4713/4713 [00:05<00:00, 880.48it/s]


INFO: Testing on /projects/p31666/zzhang/doublet-bchmk/data/fatemap_data/FM08/fatemapID/stepFourStarcodeShavedReads50.txt
Current Sample Adjusted UMI cutoff: 5
Current Sample Adjusted UMI cutoff: 11
Current Sample Adjusted UMI cutoff: 0


100%|██████████| 466/466 [00:00<00:00, 1221.86it/s]


## Test regular singlets

In [227]:
all_fatemap_read_files = grab_input_files(data_root, pattern)

dataset_dicts={}
def get_multilane_barcodes(good_data):
    multilane_barcodes = []
    for cur_barcode in good_data["cellID"].unique():
        cur_df = good_data[good_data["cellID"] == cur_barcode]
        if (len(cur_df["sampleNum"].unique()) > 1):
            multilane_barcodes.append(cur_barcode)
    return multilane_barcodes

for cur_fatemap_read_file in all_fatemap_read_files:
    print("INFO: Testing on {}".format(cur_fatemap_read_file))
    df = pd.read_csv(cur_fatemap_read_file, sep='\t')
    header = list(df.columns)
    if header[-1].lower() == "sampleNum".lower():
        header[-1] = "sampleNum"
        df.columns = header
    good_data_ls = []
    for cur_sample_num in df["sampleNum"].unique():
        cur_sample_df = df[df["sampleNum"] == cur_sample_num]
        cur_umi_adjusted_cutoff = round(umi_cutoff_ratio * cur_sample_df.shape[0])
        print("Current Sample {} Adjusted UMI cutoff: {}".format(cur_sample_num, cur_umi_adjusted_cutoff))
        cur_freq_df = cur_sample_df.groupby(['cellID', 'BC50StarcodeD8', 'sampleNum']).size().reset_index()
        cur_freq_df.columns = ['cellID', "fatemapID", 'sampleNum', "nUMI"]
        cur_good_data = cur_freq_df[cur_freq_df['nUMI'] >= cur_umi_adjusted_cutoff].reset_index(drop=True)
        good_data_ls.append(cur_good_data)

    good_data = pd.concat(good_data_ls)
    good_data = good_data.sort_values(by=["sampleNum", 'cellID', 'fatemapID', 'nUMI']).reset_index(drop=True)
    
    
    blacklist_barcodes = get_multilane_barcodes(good_data)
    
    all_samples = good_data['sampleNum'].unique()
    cellID2fatemap_dict = {}
    cellID2fatemap_count_dict = {}

    # initialize the count dictionaries
    for i in all_samples:
        cellID2fatemap_dict[i] = {}
        cellID2fatemap_count_dict[i] = {}

    # loop through each row
    for index, row in good_data.iterrows():
        cur_cellID = row['cellID']
        cur_fateID = row['fatemapID']
        cur_sample_num = int(row['sampleNum'])

        # only look at samples 1 and 2 for now
        # if(cur_sample_num not in all_samples):
        #     continue
        if cur_cellID not in cellID2fatemap_dict[cur_sample_num].keys():
            cellID2fatemap_dict[cur_sample_num][cur_cellID] = [cur_fateID]
            cellID2fatemap_count_dict[cur_sample_num][cur_cellID] = 1
        else:
            # only add if the fateID is not present in the dict
            if cur_fateID not in cellID2fatemap_dict[cur_sample_num][cur_cellID]:
                cellID2fatemap_dict[cur_sample_num][cur_cellID].append(cur_fateID)
                cellID2fatemap_count_dict[cur_sample_num][cur_cellID] += 1

    singlets = []
    multiplets = []
    print("Total cells: {}".format(len(good_data["cellID"])))
    for sample_id, cur_dict in cellID2fatemap_count_dict.items():
        cur_sample_count = 0
        for barcode, count in cur_dict.items():
#             if the barcode is present in more than one lane, remove it
            if barcode in blacklist_barcodes:
                continue
            if count == 1:
                cur_sample_count += 1
                singlets.append(barcode)
            else:
                multiplets.append(barcode)
        print("Sample {} singlet: {}".format(sample_id, cur_sample_count))
    print("Total Singlets: {}\nTotal Multiplets: {}".format(len(singlets), len(multiplets)))
    
    ########################
    ######## TEST ##########
    ########################
    cellIDs_test = random.sample(singlets, min(num_tests, len(singlets)))
    for cur_id in tqdm(cellIDs_test):
        cur_df = good_data[(good_data["cellID"]==cur_id)]
        cur_cellID_count = cur_df.shape[0]
        if(cur_cellID_count!=1):
            break
    
    

INFO: Testing on /projects/p31666/zzhang/doublet-bchmk/data/fatemap_data/FM04/fatemapID/stepFourStarcodeShavedReads50.txt
Current Sample 1 Adjusted UMI cutoff: 2
Current Sample 2 Adjusted UMI cutoff: 3
Total cells: 28077
Sample 1 singlet: 2222
Sample 2 singlet: 4608
Total Singlets: 6830
Total Multiplets: 6390


100%|██████████| 5000/5000 [00:07<00:00, 635.90it/s]


INFO: Testing on /projects/p31666/zzhang/doublet-bchmk/data/fatemap_data/non_cancer/fatemapID/stepFourStarcodeShavedReads50.txt
Current Sample 1 Adjusted UMI cutoff: 4
Current Sample 2 Adjusted UMI cutoff: 2
Total cells: 44493
Sample 1 singlet: 1353
Sample 2 singlet: 766
Total Singlets: 2119
Total Multiplets: 3439


100%|██████████| 2119/2119 [00:04<00:00, 433.20it/s]


INFO: Testing on /projects/p31666/zzhang/doublet-bchmk/data/fatemap_data/FM05/fatemapID/stepFourStarcodeShavedReads50.txt
Current Sample 1 Adjusted UMI cutoff: 2
Current Sample 2 Adjusted UMI cutoff: 2
Current Sample 3 Adjusted UMI cutoff: 1
Current Sample 4 Adjusted UMI cutoff: 1
Total cells: 124224
Sample 1 singlet: 6216
Sample 2 singlet: 6498
Sample 3 singlet: 1219
Sample 4 singlet: 1911
Total Singlets: 15844
Total Multiplets: 24263


...

100%|██████████| 5000/5000 [00:12<00:00, 390.21it/s]


INFO: Testing on /projects/p31666/zzhang/doublet-bchmk/data/fatemap_data/FM03/fatemapID/stepFourStarcodeShavedReads50.txt
Current Sample 1 Adjusted UMI cutoff: 12
Current Sample 2 Adjusted UMI cutoff: 12
Total cells: 13397
Sample 1 singlet: 4415
Sample 2 singlet: 3762
Total Singlets: 8177
Total Multiplets: 1930


100%|██████████| 5000/5000 [00:04<00:00, 1087.78it/s]


INFO: Testing on /projects/p31666/zzhang/doublet-bchmk/data/fatemap_data/FM08/fatemapID/stepFourStarcodeShavedReads50.txt
Current Sample 1 Adjusted UMI cutoff: 5
Current Sample 2 Adjusted UMI cutoff: 11
Current Sample 3 Adjusted UMI cutoff: 0
Total cells: 7020
Sample 1 singlet: 968
Sample 2 singlet: 2548
Sample 3 singlet: 234
Total Singlets: 3750
Total Multiplets: 1348


100%|██████████| 3750/3750 [00:02<00:00, 1663.39it/s]


## Test multilane singlets 

In [253]:
all_fatemap_read_files = grab_input_files(data_root, pattern)

dataset_dicts={}
def get_multilane_barcodes(good_data):
    multilane_barcodes = []
    for cur_barcode in good_data["cellID"].unique():
        cur_df = good_data[good_data["cellID"] == cur_barcode]
        if (len(cur_df["sampleNum"].unique()) > 1):
            multilane_barcodes.append(cur_barcode)
    return multilane_barcodes

for cur_fatemap_read_file in all_fatemap_read_files:
    print("INFO: Testing on {}".format(cur_fatemap_read_file))
    df = pd.read_csv(cur_fatemap_read_file, sep='\t')
    header = list(df.columns)
    if header[-1].lower() == "sampleNum".lower():
        header[-1] = "sampleNum"
        df.columns = header
    good_data_ls = []
    for cur_sample_num in df["sampleNum"].unique():
        cur_sample_df = df[df["sampleNum"] == cur_sample_num]
        cur_umi_adjusted_cutoff = round(umi_cutoff_ratio * cur_sample_df.shape[0])
        print("Current Sample {} Adjusted UMI cutoff: {}".format(cur_sample_num, cur_umi_adjusted_cutoff))
        cur_freq_df = cur_sample_df.groupby(['cellID', 'BC50StarcodeD8', 'sampleNum']).size().reset_index()
        cur_freq_df.columns = ['cellID', "fatemapID", 'sampleNum', "nUMI"]
        cur_good_data = cur_freq_df[cur_freq_df['nUMI'] >= cur_umi_adjusted_cutoff].reset_index(drop=True)
        good_data_ls.append(cur_good_data)

    good_data = pd.concat(good_data_ls)
    good_data = good_data.sort_values(by=["sampleNum", 'cellID', 'fatemapID', 'nUMI']).reset_index(drop=True)
    
    
    blacklist_barcodes = get_multilane_barcodes(good_data)
    
    all_samples = good_data['sampleNum'].unique()
    cellID2fatemap_dict = {}
    cellID2fatemap_count_dict = {}

    # initialize the count dictionaries
    for i in all_samples:
        cellID2fatemap_dict[i] = {}
        cellID2fatemap_count_dict[i] = {}

    # loop through each row
    for index, row in good_data.iterrows():
        cur_cellID = row['cellID']
        cur_fateID = row['fatemapID']
        cur_sample_num = int(row['sampleNum'])

        # only look at samples 1 and 2 for now
        # if(cur_sample_num not in all_samples):
        #     continue
        if cur_cellID not in cellID2fatemap_dict[cur_sample_num].keys():
            cellID2fatemap_dict[cur_sample_num][cur_cellID] = [cur_fateID]
            cellID2fatemap_count_dict[cur_sample_num][cur_cellID] = 1
        else:
            # only add if the fateID is not present in the dict
            if cur_fateID not in cellID2fatemap_dict[cur_sample_num][cur_cellID]:
                cellID2fatemap_dict[cur_sample_num][cur_cellID].append(cur_fateID)
                cellID2fatemap_count_dict[cur_sample_num][cur_cellID] += 1

    singlets = []
    multiplets = []
    print("Total cells: {}".format(len(good_data["cellID"])))
    for sample_id, cur_dict in cellID2fatemap_count_dict.items():
        cur_sample_count = 0
        for barcode, count in cur_dict.items():
#             if the barcode is present in more than one lane, remove it
            if barcode in blacklist_barcodes:
                continue
            if count == 1:
                cur_sample_count += 1
                singlets.append(barcode)
            else:
                multiplets.append(barcode)
        print("Sample {} singlet: {}".format(sample_id, cur_sample_count))
    print("Total Singlets: {}\nTotal Multiplets: {}".format(len(singlets), len(multiplets)))
    
    fatemapID_dict = {}
    fatemapID_count_dict = {}

    for cur_multiplet in multiplets:
        cur_df = good_data[good_data["cellID"] == cur_multiplet]
        # only consider when more than 1 fate barcode appears
        n_unique_fatemap_barcodes = len(cur_df["fatemapID"].unique())
        if n_unique_fatemap_barcodes < 2:
            continue
        # create unique key for each fatemap barcode ID
        fatemapID_combo = "_".join(sorted(cur_df["fatemapID"].unique()))
        if fatemapID_combo not in fatemapID_dict:
            fatemapID_dict[fatemapID_combo] = [cur_multiplet]
            fatemapID_count_dict[fatemapID_combo] = 1
        else:
            fatemapID_dict[fatemapID_combo].append(cur_multiplet)
            fatemapID_count_dict[fatemapID_combo] += 1

    two_barcode_singlets_count = 0
    two_barcode_singlets = []
    for key, value in fatemapID_count_dict.items():
        if value >= 2:
            two_barcode_singlets_count += value
            two_barcode_singlets.extend(fatemapID_dict[key])
            if fatemapID_dict[key] in two_barcode_singlets:
                print(fatemapID_dict[key])
    ########################
    ######## TEST ##########
    ########################
    for cur_two_barcode_singlet in tqdm(two_barcode_singlets):
        cur_df = good_data[good_data["cellID"] == cur_two_barcode_singlet]
        cur_fatemap_barcodes = "_".join(sorted(cur_df["fatemapID"].unique()))
        cur_recovered_singlets = fatemapID_dict[cur_fatemap_barcodes]
        cur_recovered_df = good_data[good_data["cellID"].isin(fatemapID_dict[fatemapID_combo])]
        for cur_recovered_singlet in cur_recovered_df["cellID"].unique():
            cur_recovered_df_cur_singlet = cur_recovered_df[cur_recovered_df["cellID"]==cur_recovered_singlet]
            cur_recovered_singlet_fatemap_combo = "_".join(sorted(cur_recovered_df_cur_singlet["fatemapID"].unique()))
            if(cur_recovered_singlet_fatemap_combo!=cur_fatemap_barcodes):
                break

INFO: Testing on /projects/p31666/zzhang/doublet-bchmk/data/fatemap_data/FM04/fatemapID/stepFourStarcodeShavedReads50.txt
Current Sample 1 Adjusted UMI cutoff: 2
Current Sample 2 Adjusted UMI cutoff: 3
Total cells: 28077
Sample 1 singlet: 2222
Sample 2 singlet: 4608
Total Singlets: 6830
Total Multiplets: 6390


100%|██████████| 2935/2935 [00:10<00:00, 286.94it/s]


INFO: Testing on /projects/p31666/zzhang/doublet-bchmk/data/fatemap_data/non_cancer/fatemapID/stepFourStarcodeShavedReads50.txt
Current Sample 1 Adjusted UMI cutoff: 4
Current Sample 2 Adjusted UMI cutoff: 2
Total cells: 44493
Sample 1 singlet: 1353
Sample 2 singlet: 766
Total Singlets: 2119
Total Multiplets: 3439


100%|██████████| 271/271 [00:01<00:00, 269.66it/s]


INFO: Testing on /projects/p31666/zzhang/doublet-bchmk/data/fatemap_data/FM05/fatemapID/stepFourStarcodeShavedReads50.txt
Current Sample 1 Adjusted UMI cutoff: 2
Current Sample 2 Adjusted UMI cutoff: 2
Current Sample 3 Adjusted UMI cutoff: 1
Current Sample 4 Adjusted UMI cutoff: 1
Total cells: 124224
Sample 1 singlet: 6216
Sample 2 singlet: 6498
Sample 3 singlet: 1219
Sample 4 singlet: 1911
Total Singlets: 15844
Total Multiplets: 24263


...

100%|██████████| 3875/3875 [00:26<00:00, 148.26it/s]


INFO: Testing on /projects/p31666/zzhang/doublet-bchmk/data/fatemap_data/FM03/fatemapID/stepFourStarcodeShavedReads50.txt
Current Sample 1 Adjusted UMI cutoff: 12
Current Sample 2 Adjusted UMI cutoff: 12
Total cells: 13397
Sample 1 singlet: 4415
Sample 2 singlet: 3762
Total Singlets: 8177
Total Multiplets: 1930


100%|██████████| 664/664 [00:01<00:00, 461.11it/s]


INFO: Testing on /projects/p31666/zzhang/doublet-bchmk/data/fatemap_data/FM08/fatemapID/stepFourStarcodeShavedReads50.txt
Current Sample 1 Adjusted UMI cutoff: 5
Current Sample 2 Adjusted UMI cutoff: 11
Current Sample 3 Adjusted UMI cutoff: 0
Total cells: 7020
Sample 1 singlet: 968
Sample 2 singlet: 2548
Sample 3 singlet: 234
Total Singlets: 3750
Total Multiplets: 1348


100%|██████████| 1050/1050 [00:27<00:00, 38.37it/s] 


## Test UMI singlets

In [289]:
singlets_stat_ls=[]

for cur_fatemap_read_file in all_fatemap_read_files:
    cur_singlet_stats = []
    df = pd.read_csv(cur_fatemap_read_file, sep='\t')
    
    # some sampleNum is written as SampleNum
    # force conversion
    header = list(df.columns)
    if header[-1].lower() == "sampleNum".lower():
        header[-1] = "sampleNum"
        df.columns = header

    df_sum = df['sampleNum'].value_counts()
    all_samples = df['sampleNum'].unique()
    print("INFO: Raw data counts")
    print(df_sum)

    # adjust UMI cutoff based on reads
    # drop cells that do not pass UMI cutoff
    good_data_ls = []
    for cur_sample_num in df["sampleNum"].unique():
        cur_sample_df = df[df["sampleNum"] == cur_sample_num]
        cur_umi_adjusted_cutoff = round(umi_cutoff_ratio * cur_sample_df.shape[0])
        print("Current Sample Adjusted UMI cutoff: {}".format(cur_umi_adjusted_cutoff))
        cur_freq_df = cur_sample_df.groupby(['cellID', 'BC50StarcodeD8', 'sampleNum']).size().reset_index()
        cur_freq_df.columns = ['cellID', "fatemapID", 'sampleNum', "nUMI"]
        cur_good_data = cur_freq_df[cur_freq_df['nUMI'] >= cur_umi_adjusted_cutoff].reset_index(drop=True)
        good_data_ls.append(cur_good_data)
    good_data = pd.concat(good_data_ls)
    good_data = good_data.sort_values(by=["sampleNum", 'cellID', 'fatemapID', 'nUMI']).reset_index(drop=True)

    # get fatemap barcode count for each cellID
    cellID2fatemap_dict, cellID2fatemap_count_dict = \
        generate_fatemap_barcode_counts_for_cellID(good_data)

    # get multilane barcodes
    multilane_barcodes = get_multilane_barcodes(good_data)

    # get first round of results
    singlets, multiplets = define_singlets_and_multiplets_based_on_fatemapID_counts(cellID2fatemap_count_dict,
                                                                                    good_data,
                                                                                    multilane_barcodes)
    cur_singlet_stats.append(len(singlets))
    # salvage false multiplets
    fatemapID_dict, fatemapID_count_dict = generate_fatemapID_combo(multiplets, \
                                                                    good_data)

    two_barcode_singlets = extract_two_fatemapID_singlets(fatemapID_dict, \
                                                          fatemapID_count_dict)
    
    cur_singlet_stats.append(len(two_barcode_singlets))
    # update singlets and multiplets
    singlets_step2 = list(set(singlets).union(set(two_barcode_singlets)))
    multiplets_step2 = list(set(multiplets).difference(set(two_barcode_singlets)))
    
    
    UMI_thres_singlets = []
    for cur_multiplet in multiplets_step2:
        cur_df = good_data[good_data["cellID"] == cur_multiplet]
        cur_UMI_counts = cur_df['nUMI'].values
        cur_median_UMI = np.median(cur_UMI_counts)
        if (np.max(cur_UMI_counts) - cur_median_UMI) > umi_diff_threshold:
            # check if there is more than 1 dominant
            dominant_count = 0
            for cur_UMI in cur_UMI_counts:
                cur_diff = cur_UMI - cur_median_UMI
                if cur_diff > umi_diff_threshold or cur_UMI > dominant_threshold:
                    dominant_count += 1

            if dominant_count == 1:
                UMI_thres_singlets.append(cur_multiplet)
                break
    cur_singlet_stats.append(len(UMI_thres_singlets))
    singlets_step3 = list(set(singlets_step2).union(set(UMI_thres_singlets)))
    multiplets_step3 = list(set(multiplets_step2).difference(set(UMI_thres_singlets)))
    print("Total Singlets: {}\nTotal Multiplets: {}".format(len(singlets_step3), len(multiplets_step3)))
    cur_singlet_stats.append(len(singlets_step3))
    cur_singlet_stats.append(len(multiplets_step3))
    singlets_stat_ls.append(cur_singlet_stats)
    ########################
    ######## TEST ##########
    ########################
    
    for cur_umi_thres_singlet in tqdm(UMI_thres_singlets, desc="Benchmarking"):
        cur_df = good_data[good_data["cellID"] == cur_umi_thres_singlet]
        cur_UMIs = cur_df['nUMI'].values
        cur_median = np.median(cur_UMIs)
        cur_dominant_umi_num = sum(cur_UMIs-dominant_threshold>0)
        if(cur_dominant_umi_num!=1):
            break  

INFO: Raw data counts
2    404787
1    313223
Name: sampleNum, dtype: int64
Current Sample Adjusted UMI cutoff: 2
Current Sample Adjusted UMI cutoff: 3


100%|██████████| 28077/28077 [00:01<00:00, 27089.01it/s]


Total cells: 28077
Sample 1 singlet: 2222
Sample 2 singlet: 4608
Total Singlets: 6830
Total Multiplets: 6390
All singlets identified are unique? True
Total Singlets: 9766
Total Multiplets: 3454


Benchmarking: 100%|██████████| 1/1 [00:00<00:00, 330.16it/s]


INFO: Raw data counts
1    475505
2    315727
Name: sampleNum, dtype: int64
Current Sample Adjusted UMI cutoff: 4
Current Sample Adjusted UMI cutoff: 2


...

Benchmarking: 0it [00:00, ?it/s]


INFO: Raw data counts
2    1454787
1     614619
3      11716
Name: sampleNum, dtype: int64
Current Sample Adjusted UMI cutoff: 5
Current Sample Adjusted UMI cutoff: 11
Current Sample Adjusted UMI cutoff: 0


100%|██████████| 7020/7020 [00:00<00:00, 26102.99it/s]


Total cells: 7020
Sample 1 singlet: 968
Sample 2 singlet: 2548
Sample 3 singlet: 234
Total Singlets: 3750
Total Multiplets: 1348
All singlets identified are unique? True
Total Singlets: 4801
Total Multiplets: 297


Benchmarking: 100%|██████████| 1/1 [00:00<00:00, 1172.25it/s]


In [305]:
singlets_stat_df = pd.DataFrame(cur_singlet_stats).T
singlets_stat_df.columns = ["single_fatemap_barcode_singlets", "multi_fatemap_barcode_singlets", \
                           "dominant_umi_barcode_singlets", "total_singlets", "total_undetermined"]
singlets_stat_df["total_cells"]=singlets_stat_df["total_singlets"]+singlets_stat_df["total_undetermined"]
singlets_stat_df["dataset"] = cur_fatemap_read_file.split("/")[-3]

In [306]:
singlets_stat_df

,single_fatemap_barcode_singlets,multi_fatemap_barcode_singlets,dominant_umi_barcode_singlets,total_singlets,total_undetermined,total_cells,dataset
0,3750,1050,1,4801,297,5098,FM08


In [304]:
cur_fatemap_read_file.split("/")[-3]

'FM08'

In [280]:
cur_median

2.0

In [285]:
sum(cur_UMIs-umi_diff_threshold>0)

1